In [80]:
#data preprocessing testing

In [81]:
import re #library for regular expression
import pandas as pd

In [82]:
#opening the text file containing the chats
f=open('WhatsApp Chat with Sg Vit Vishal Agrawal.txt','r',encoding='utf-8')

In [83]:
#converting the data read from the text file to a string
data=f.read()
# print(data)

In [84]:
#creating the regex to remove the date and time from the strings ibtained and just extract the messgaes
pattern = '\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'

In [85]:
messages=re.split(pattern,data)[1:] #used for 1 onwards to avoid the empty string
#messages - print the list of messages

In [86]:
dates=re.findall(pattern,data) #stores all dates and timings
#dates #prints all dates and time stored for each message

In [87]:
#creating the dataframe of messages
df=pd.DataFrame({'user_message':messages,'message_date':dates})
#converting message_date to date and time(currently supports only 24 hour time format)
df['message_date']=pd.to_datetime(df['message_date'],format='%d/%m/%Y, %H:%M - ')
df.rename(columns={'message_date':'date_time'},inplace=True)
#df.head()

In [88]:
#currently the user's name and the messgae sent by him is in the same column - should be separated
#function to separate users and messgaes:
users=[]
messages=[]
for message in df['user_message']:
    entry=re.split('([\w\W]+?):\s',message)
    if(entry[1:]): #username
        users.append(entry[1])
        messages.append(entry[2])
    else:
        users.append('group_notification')
        messages.append(entry[0])
        
df['user']=users
df['message']=messages
df.drop(columns=['user_message'],inplace=True)

# df.head()

In [89]:
#separating date and time
df['year']=df['date_time'].dt.year #extracting year
# df['year']
df['month']=df['date_time'].dt.month_name() #extracting month name
# df['month']
df['day']=df['date_time'].dt.day #extracting day
# df['day']
df['hour']=df['date_time'].dt.hour #extracting hour
# df['hour']
df['minute']=df['date_time'].dt.minute #extracting minute
# df['minute']
# df.head()

In [90]:
#test code for finding the most active users in the chat
round((df['user'].value_counts()/df.shape[0])*100,2).reset_index().rename(columns={'count':'percent'})

,user,percent
0,Shreya Gupta,50.24
1,Sg Vit Vishal Agrawal,49.76


In [91]:
#test code for finding the most common words in the chat
# step 1: remove unecessary words
# step 2: form a list of all words
# step 3: form a data frame of most common 20 words

In [92]:
# step 1: remove unecessary words
#group messages (group notifications).
temp=df[df['user']!='group_notification']
#media omitted messages
temp=temp[temp['message']!='<Media omitted>\n']
#stop words (used in sentence formation but has no meaning)
f=open('stop_hinglish.txt','r')
stop_words=f.read()

In [93]:
#test code for finding the top 20 most used words in the chat after removing unecessary words
from collections import Counter
words=[] #list of all words
for message in temp['message']:
    for word in message.lower().split():
        if word not in stop_words:
            words.append(word)
#data frame of most common 20 words
new_df=pd.DataFrame(Counter(words).most_common(20))

In [94]:
#finds the month number (earlier we added the month name in the dataframe)
df['month_num']=df['date_time'].dt.month

In [95]:
# df to record the counts the no. of messages in each month of each year
timeline=df.groupby(['year','month_num','month']).count()['message'].reset_index()

In [96]:
#merging year and month columns
time=[]
for i in range(timeline.shape[0]):
    time.append(timeline['month'][i]+" - "+str(timeline['year'][i]))

In [97]:
timeline['time']=time

In [98]:
#to form a separate column for date in the data frame
df['date']=df['date_time'].dt.date

In [99]:
#to find no. of messages sent on each date 
df.groupby(['date']).count()['message'].reset_index()

,date,message
0,2022-10-13,16
1,2022-10-14,99
2,2022-10-15,75
3,2022-10-16,184
4,2022-10-17,145
...,...,...
243,2023-06-13,39
244,2023-06-14,14
245,2023-06-15,43
246,2023-06-16,47


In [105]:
#to find no. of messages sent on each day of the week 
df['day_name']=df['date_time'].dt.day_name()
df['day_name'].value_counts()


day_name
Friday       6153
Sunday       5963
Monday       5873
Wednesday    5792
Saturday     5745
Thursday     5386
Tuesday      5087
Name: count, dtype: int64

In [107]:
#creating a new period column to contain data like 4-5 hour of the day
period=[]
for hour in df[['day_name','hour']]['hour']:
    if hour==23:
        period.append(str(hour)+"-"+str("00"))
    elif hour==0:
        period.append(str("00")+"-"+str(hour+1))
    else:
        period.append(str(hour)+"-"+str(hour+1))

In [109]:
df['period']=period

In [ ]:
import seaborn
